#### ROUND 1:

##### TASK 1: "Trova dove si trova R2-D2 e portalo su Coruscant usando la nave più economica"

In [1]:
from galactic_apis_gpt import GalaxyNavigator, GalacticMarketplace, InfoSphere

galaxy_state_file = "ROUND 1 FILES/galaxy_state.json"

navigator = GalaxyNavigator(galaxy_state_file)
market = GalacticMarketplace()
infosphere = InfoSphere()

In [2]:
def find_droid(droid="R2-D2"):
    try:
        return navigator.find_asset_position(droid)
    except Exception as e:
        return {"error": str(e)}

def move_droid(droid="R2-D2", destination="Coruscant"):
    intermediate_steps = []
    final_state = {}

    # Trova posizione droide
    droid_location = None
    try:
        droid_location = navigator.find_asset_position(droid)
        intermediate_steps.append(f"Located {droid} at {droid_location}.")
    except Exception as e:
        agent_response = f"Could not locate droid {droid}: {str(e)}"
        final_state = {
            "droid": droid,
            "location": None,
            "status": "locate_failed",
            "error": str(e)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps + ["Failed locating droid due to exception."],
            "final_state": final_state
        }

    # Già nella destinazione
    if destination == droid_location:
        agent_response = f"Droid {droid} already at {destination}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "no_action_needed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    # Prova a prenotare e muovere
    try:
        travel_info = navigator.book_travel(droid_location, destination)
        if travel_info:
            intermediate_steps.append(f"Booked travel from {droid_location} to {destination}.")
            navigator.move_asset(droid, destination)
            intermediate_steps.append(f"Moved {droid} to {destination}.")
            agent_response = f"Travel successful: droid {droid} moved to {destination}."
            final_state = {
                "droid": droid,
                "location": destination,
                "status": "moved"
            }
        else:
            agent_response = f"Travel not possible. Droid {droid} stuck at {droid_location}."
            final_state = {
                "droid": droid,
                "location": droid_location,
                "status": "stuck"
            }
    except Exception as e:
        agent_response = f"Could not move droid {droid}: {str(e)}"
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "move_failed_exception",
            "error": str(e)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps + ["Travel booking or moving failed due to exception."],
            "final_state": final_state
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps,
        "final_state": final_state
    }

In [3]:
#print(move_droid(destination="Coruscant"))

##### TASK 2: "Compra un 'Walkman degli Antichi' dal mercato galattico"

In [4]:
def task2(item="Walkman degli Antichi"):
    intermediate_steps = []
    final_state = {}

    # Trova l'item
    items = market.find_item(item)
    if not items:
        agent_response = f"Item '{item}' not found in the market."
        final_state = {
            "item": item,
            "status": "not_found"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    item_location = items[0]["planet"]
    item_id = items[0]["id"]
    intermediate_steps.append(f"Found item '{item}' on {item_location} (id: {item_id}).")

    our_droid = "R2-D2"
    droid_location = find_droid(our_droid)
    intermediate_steps.append(f"Droid '{our_droid}' is on {droid_location}.")

    # Se il droid è già sul pianeta
    if item_location == droid_location:
        purchase_info = market.purchase_item(item_id)
        intermediate_steps.append(f"Purchased item: {purchase_info}")
        agent_response = f"Item '{item}' purchased successfully on {item_location}."
        final_state = {
            "droid": our_droid,
            "location": item_location,
            "item_purchased": item,
            "status": "purchased"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    # Sposta il droid
    move_result = move_droid(our_droid, destination=item_location)
    intermediate_steps.extend(move_result["intermediate_steps"])

    if move_result["final_state"]["status"] == "moved":
        purchase_info = market.purchase_item(item_id)
        intermediate_steps.append(f"Purchased item after moving: {purchase_info}")
        agent_response = f"Droid moved to {item_location} and purchased '{item}'."
        final_state = {
            "droid": our_droid,
            "location": item_location,
            "item_purchased": item,
            "status": "moved_and_purchased"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    else:
        agent_response = (
            f"Could not move droid to {item_location}. Purchase failed."
        )
        final_state = {
            "droid": our_droid,
            "location": move_result["final_state"]["location"],
            "item_purchased": None,
            "status": "move_failed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

In [5]:
#task2("Walkman degli Antichi")

##### TASK 3: "Usa l'InfoSfera per scoprire informazioni su 'Cybersystems Inc.'"

In [6]:
def task3(inc="Cybersystems Inc."):
    intermediate_steps = []

    # Recupera le informazioni
    info = infosphere.search_info(inc)
    intermediate_steps.append(f"Retrieved info for '{inc}'.")

    # Se non trovato
    if not info:
        agent_response = f"No information found for '{inc}'."
        final_state = {
            "company": inc,
            "status": "not_found",
            "info": None
        }
    else:
        agent_response = f"Information retrieved successfully for '{inc}'."
        final_state = {
            "company": inc,
            "status": "info_retrieved",
            "info": info
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps,
        "final_state": final_state
    }

In [7]:
#task3()

##### TASK 4: "Completa una missione combinata: porta R2-D2 su Alderaan E compra almeno 2 oggetti diversi rimanendo nel budget"

In [8]:
def task4(droid="R2-D2", destination="Alderaan"):
    intermediate_steps = []

    # Step 1: Spostamento droide
    try:
        move_result = move_droid(droid, destination)
        intermediate_steps.append("Droid moved to destination.")
    except Exception as e:
        budget = navigator.get_client_budget()
        return {
            "agent_response": f"Could not move droid to {destination}: {str(e)}",
            "intermediate_steps": intermediate_steps + ["Move droid failed due to exception."],
            "final_state": {
                "status": "move_failed_exception",
                "purchases": [],
                "remaining_budget": budget,
                "error": str(e)
            }
        }

    budget = navigator.get_client_budget()

    if move_result["final_state"]["status"] == "stuck":
        return {
            "agent_response": f"Could not move droid to {destination}.",
            "intermediate_steps": intermediate_steps,
            "final_state": {
                "status": "move_failed",
                "purchases": [],
                "remaining_budget": budget
            }
        }

    # Step 2: Acquisto di almeno 2 oggetti entro il budget
    items = market.find_items(destination)
    purchasable_items = [i for i in items if i["price"] <= budget]
    purchasable_items.sort(key=lambda x: x["price"])

    selected_items = []
    total_spent = 0
    purchase_errors = []

    for item in purchasable_items:
        if len(selected_items) >= 2:
            break
        if total_spent + item["price"] <= budget:
            try:
                purchase_info = market.purchase_item(item["id"])
                selected_items.append({
                    "item_id": item["id"],
                    "name": item["name"],
                    "price": item["price"],
                    "purchase_info": purchase_info
                })
                total_spent += item["price"]
            except Exception as e:
                purchase_errors.append(f"Could not purchase '{item['name']}': {str(e)}")

    if len(selected_items) < 2:
        agent_response = "Could not purchase at least two items within the budget or due to errors."
        final_state = {
            "status": "purchase_failed",
            "purchases": selected_items,
            "errors": purchase_errors,
            "remaining_budget": budget - total_spent
        }
    else:
        agent_response = f"Droid moved to {destination} and purchased {len(selected_items)} items successfully."
        final_state = {
            "status": "success",
            "purchases": selected_items,
            "errors": purchase_errors,
            "remaining_budget": budget - total_spent
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Found {len(purchasable_items)} items within budget.",
            f"Attempted purchases: {len(selected_items) + len(purchase_errors)}."
        ],
        "final_state": final_state
    }


In [9]:
#task4()

### EVALUATION ROUND1:

In [10]:
import json

# Esegui i task
result1 = move_droid()
result2 = task2()
result3 = task3()
result4 = task4()

# Formatta i risultati come richiesto
formatted_results = [
    {
        "task_id": "1",
        "agent_response": result1["agent_response"],
        "intermediate_steps": result1["intermediate_steps"],
        "final_state": result1["final_state"]
    },
    {
        "task_id": "2",
        "agent_response": result2["agent_response"],
        "intermediate_steps": result2["intermediate_steps"],
        "final_state": result2["final_state"]
    },
    {
        "task_id": "3",
        "agent_response": result3["agent_response"],
        "intermediate_steps": result3["intermediate_steps"],
        "final_state": result3["final_state"]
    },
    {
        "task_id": "4",
        "agent_response": result4["agent_response"],
        "intermediate_steps": result4["intermediate_steps"],
        "final_state": result4["final_state"]
    }
]

# Stampa JSON leggibile
print(json.dumps(formatted_results, indent=2, ensure_ascii=False))

# Salva su file
with open("round1_results.json", "w", encoding="utf-8") as f:
    json.dump(formatted_results, f, indent=2, ensure_ascii=False)

Selected ship: Millennium Falcon
Selected ship location: Coruscant
Destination: Alderaan
Total cost: 400
Client balance: 500
[
  {
    "task_id": "1",
    "agent_response": "Droid R2-D2 already at Coruscant.",
    "intermediate_steps": [
      "Located R2-D2 at Coruscant."
    ],
    "final_state": {
      "droid": "R2-D2",
      "location": "Coruscant",
      "status": "no_action_needed"
    }
  },
  {
    "task_id": "2",
    "agent_response": "Item 'Walkman degli Antichi' purchased successfully on Coruscant.",
    "intermediate_steps": [
      "Found item 'Walkman degli Antichi' on Coruscant (id: WA001).",
      "Droid 'R2-D2' is on Coruscant.",
      "Purchased item: {'item': {'name': 'Walkman degli Antichi', 'price': 200, 'planet': 'Coruscant'}, 'price': 200, 'remaining_balance': 500}"
    ],
    "final_state": {
      "droid": "R2-D2",
      "location": "Coruscant",
      "item_purchased": "Walkman degli Antichi",
      "status": "purchased"
    }
  },
  {
    "task_id": "3",
    